# **Predicción Ventas de Yamaha 2024-2025**

## **First Steps**

Set init imports and variables

In [ ]:
# Import needed libraries
import pandas
import numpy
import seaborn
from matplotlib import pyplot
%matplotlib inline

In [ ]:
# Set Project params
data_filter = 'modelo' # [asesor, modelo]
data_values = 'cantidad' # [cantidad, costo]

data_time_start = '2019' # =>  2022-01-01
data_time_end = '2024' # =>  2024-01-01

show_plots = False

In [ ]:
# Set the variables according Project params
filter_mappings = {
   'asesor': 'nom_asesor',
   'modelo': 'modelo',
   'clasificacion': 'clasificacion'
}
values_mappings = {
   'cantidad': {
      'name': 'cantidad',
      'type': int
   },
   'costo': {
      'name': 'costo_unitario',
      'type': float
   }
}

selected_filter = filter_mappings.get(data_filter)
selected_value = values_mappings.get(data_values)

## **Get and Prepare Data**

Data connection, cleaning, normalize, filter and sort.

In [ ]:
# Set data files paths
import os

query_path = '../assets/query.sql'
normalize_path = f'../assets/{data_filter}.csv'

data_path = '../assets/data.csv'
data_clean_path = '../assets/data_cleaned.csv'
data_normalized_path = '../assets/data_normalized.csv'
data_filter_path = '../assets/data_filtered.csv'

### Connect

In [ ]:
# Get connection env variables 
if not os.path.exists(data_path):

   import dotenv

   dotenv.load_dotenv( )

   DRIVER = 'ODBC Driver 18 for SQL Server'
   SERVER = os.getenv('PROJECT_SERVER')
   DATABASE = os.getenv('PROJECT_DATABASE')
   USERNAME = os.getenv('PROJECT_USERNAME')
   PASSWORD = os.getenv('PROJECT_PASSWORD')

In [ ]:
# Fetch data from DB
if not os.path.exists(data_path):

   # import pyodbc
   import sqlalchemy
   
   # Get SQL query from file
   with open(query_path) as file:
      sql_query = file.read()

   # PyODBC connection
   # pyodbc_connection_string = f'DRIVER={DRIVER};SERVER={SERVER};DATABASE={DATABASE};UID={USERNAME};PWD={PASSWORD};TrustServerCertificate=YES;'
   # pyodbc_connection = pyodbc.connect(pyodbc_connection_string)

   # SQLAlquemy connection
   sqlalchemy_connection_string = f'mssql+pyodbc://{USERNAME}:{PASSWORD}@{SERVER}/{DATABASE}?driver={DRIVER}&TrustServerCertificate=yes'
   sqlalchemy_engine = sqlalchemy.create_engine(sqlalchemy_connection_string)

   # Excecute query with pandas
   query = pandas.read_sql_query(sql_query, sqlalchemy_engine)

In [ ]:
# Save data
if not os.path.exists(data_path):   
   
   fetched_data = pandas.DataFrame(query)
   fetched_data.to_csv(data_path, index=False, header=True, sep=";")

### Clean

In [ ]:
# Load data to clean
with open(data_path, 'r') as file:
   data = file.read().split('\n')

   headers = data.pop(0)
   clean_data = '\n'.join(data)

   print(data[0] +'\n'+ data[17792] +'\n'+ data[8667] +'\n'+ data[30405])

In [ ]:
# Clean data by removing extra characters

# decimals to int values
clean_data = clean_data.replace('.0;', ';')
# extra commas
clean_data = clean_data.replace(',', '')
# double spaces at start and end of any cell
clean_data = clean_data.replace('; ', ';')
clean_data = clean_data.replace(' ;', ';')
# double spaces at middle of any cell
clean_data = clean_data.replace('\n', '_')
clean_data = ' '.join(clean_data.split())
clean_data = clean_data.replace('_', '\n')
# extra quotation marks
clean_data = clean_data.replace('"', '')


clean_data = clean_data.split('\n')
print(clean_data[0] +'\n'+ clean_data[17792] +'\n'+ clean_data[8667] +'\n'+ clean_data[30405])
clean_data = '\n'.join(clean_data)

In [ ]:
# Save cleaned data
headers = headers.replace(';', ',')
clean_data = clean_data.replace(';', ',')

with open(data_clean_path, 'w') as file:
   file.write(headers + '\n' + clean_data)

### Normalize

In [ ]:
# Load data to normalize
if os.path.exists(normalize_path):
   normalize_data = pandas.read_csv(data_clean_path)
   
normalize_data.head()

In [ ]:
# Load normalization params
if os.path.exists(normalize_path):

   with open(normalize_path, mode='r', encoding='utf-8') as file:
      norms = file.read().replace('\n', ',').split(',')
      norms = norms[2:]
      norms = numpy.reshape(norms, (-1, 2))

   # Convert normalization csv to dict
   normalization = {}
   for model, norm_value in norms:
      normalization[model] = norm_value

normalization

In [ ]:
# Insert normalized data in a new Column
if os.path.exists(normalize_path):

   normalize_data['modelo'] = normalize_data['des_modelo'].apply(lambda x: normalization.get(x))
normalize_data.head()

In [ ]:
# Save normalized data
if os.path.exists(normalize_path):
   
   normalize_data.to_csv(data_normalized_path, index=False)

### Filter

In [ ]:
# Load data to filter
filter_data = pandas.read_csv(data_normalized_path, parse_dates=['fecha'], date_format='%Y-%m-%d')

In [ ]:
# Delete unnecesary columns
filter_data = filter_data.drop(columns=['sw', 'bodega', 'ident_asesor', 'ident_cliente', 'nom_cliente', 'utilidad', 'financiera', 'dias_inv', 'doc_ref'])
filter_data.head()

In [ ]:
# Filter by time-range Project params
filter_data = filter_data[(data_time_start <= filter_data['fecha']) & (filter_data['fecha']<= data_time_end)]
filter_data

In [ ]:
# Group dataframe by ['fecha'] as primary and [selected_filter] as secondary
filter_data_group = filter_data.groupby([pandas.Grouper(key='fecha', freq='D', sort=True), selected_filter])[selected_value['name']].sum()
filter_data_group

In [ ]:
# Convert group series in a new dataframe
data_filtered = filter_data_group.unstack(level=1)
data_filtered.head()

In [ ]:
# Fill NaN and format int columns
data_filtered = data_filtered.fillna(0)
data_filtered = data_filtered.astype(selected_value['type'])
data_filtered.head()

In [ ]:
# Create 'TOTAL' by Time
data_filtered['TOTAL'] = data_filtered.sum(axis='columns')
# data_filtered.loc['Total']= data_filtered.sum()

data_filtered

In [ ]:
# Save filtered_data
data_filtered.to_csv(data_filter_path, date_format='%Y-%m-%d')

### Sort

In [ ]:
# Group dataframe by ['modelo'] as primary and ['des_modelo'] as secondary
sort_data_group = filter_data.groupby(['modelo', 'des_modelo'])[selected_value['name']].sum()
sort_data_group

In [ ]:
# Sort Items by Value
sort_data = filter_data.groupby(selected_filter)[selected_value['name']].sum()
sort_data = sort_data.reset_index()
sort_data = sort_data.sort_values(by='cantidad', ascending=False)
sort_data

## **Display Data**

Display sample graphs of the prepared data

### Show

In [ ]:
# Load data
data = pandas.read_csv('../assets/data_filtered.csv', parse_dates=['fecha'], date_format='%Y-%m-%d', dtype=selected_value['type'])

# Extract Items and Time
items = list(data.iloc[:, 1:-1].keys())
time = numpy.asarray(data['fecha'], dtype='datetime64[s]')

data.head()

In [ ]:
# Items all-in-one plot
if show_plots:
   
   figure, ax = pyplot.subplots(figsize=(12, 4))

   ax.plot(time, data[items], lw=1)
   ax.tick_params(axis='x', labelrotation=0)
   ax.set_title(f'No. de Ventas {data_time_start} - {data_time_end}')
   ax.set_xlabel('Fecha')
   ax.set_ylabel('Ventas')
   ax.margins(x=0.03, y=0.02)
   ax.grid()

   figure.tight_layout()

In [ ]:
# Top-10 Items - filtered data
items_top_10 = list( sort_data['modelo'].head(10) )

# Config plots for this Items subgroup
cols = 2
rows = 5
size = (10, 8)
y_limit = (0, data[items].max().max())
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']

In [ ]:
# Top-10 items same-scaled separated plots
if show_plots:
    figure_a, axes_a = pyplot.subplots(nrows=rows, ncols=cols, figsize=size)

    for index, item in enumerate(items_top_10):
        ax = axes_a[int(index/cols), int(index%cols)]
        ax.plot(time, data[item], label=item, color=colors[int(index%10)], lw=1)
        ax.tick_params(axis='x', labelrotation=0)
        ax.set(ylim=y_limit)
        ax.legend()
        # ax.grid()

    figure_a.tight_layout()

### Relations

In [ ]:
# Top-10 First & Last items - filtered data 
items_first_last = [ items_top_10[0], items_top_10[-1] ]

In [ ]:
# Top-10 First & Last items VS total sales relation - filtered data 
if show_plots:
   figure_b, axes_b = pyplot.subplots(nrows=2, ncols=1, figsize=(14, 7))

   for index, key in enumerate(items_first_last):
      ax = axes_b[index]
      ax.plot(time, data['TOTAL'], label='Total', lw=1)
      ax.plot(time, data[key], label=key, lw=1)
      ax.tick_params(axis='x', labelrotation=0)
      ax.margins(x=0.03, y=0.04)
      # ax.grid()
      ax.set(
         title=f'{key}',
         # xlabel='Fecha', 
         # ylabel='No. de Ventas',
      )
      ax.legend()

   figure_b.tight_layout()